## Grad Cam

In [ ]:
import torch
import torch.nn as nn
from torchvision import models

from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from PIL import Image
from sklearn.metrics import confusion_matrix
import seaborn as sns


classes = ['normal', 'abnormal']

In [ ]:
from utils.dataset import BodyDataset


In [ ]:
# resnet18
model = models.resnet18()
in_feature = model.fc.in_features
model.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = torch.nn.Linear(in_features=in_feature, out_features=2, bias=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model.load_state_dict(torch.load('35_350_result/loss_0.32_resnet18_v6.pt'))

model.to(device)
model.eval()
print('model loaded')